In [12]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import shutil
from PIL import Image  # Import the Image module from PIL


In [25]:
# get info form images:
def get_info_images(images_dir):
    # List subdirectories (assuming "NORMAL" and "PNEUMONIA")
    subdirectories = [os.path.join(images_dir, sub_dir) for sub_dir in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, sub_dir))]

    #summary to return
    images_size=[]

    # Iterate through subdirectories
    for sub_dir in subdirectories:
        # Get class label (subdirectory name)
        class_label = os.path.basename(sub_dir)

        # List image files in the subdirectory
        image_files = [f for f in os.listdir(sub_dir) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'))]

        # Iterate through the image files and print their information
        for image_file in image_files:
            image_path = os.path.join(sub_dir, image_file)
            try:
                with Image.open(image_path) as img:
                    # Get image properties
                    width, height = img.size
                    format = img.format
                    mode = img.mode

                    images_size.append([width,height])

                    # Print image information
                    #print(f"Class: {class_label}")
                    #print(f"Image: {image_file}")
                    #print(f"Dimensions: {width}x{height}")
                    #print(f"Format: {format}")
                    #print(f"Mode: {mode}\n")
            except Exception as e:
                print(f"Error processing {image_file}: {e}")
    
    return images_size




In [38]:
images_sizes=get_info_images('Dataset/train')

from collections import Counter

# Convert the list of lists to a tuple of tuples for hashability
values_tuples = tuple(tuple(val) for val in images_sizes)

# Count occurrences of each value in the list
value_counts = Counter(values_tuples)

# Find the most frequent value(s)
max_count = max(value_counts.values())
modes_tuples = [value for value, count in value_counts.items() if count == max_count]

# Convert the modes back to lists
modes = [list(val) for val in modes_tuples]

print(f"The most frequent value is: {modes}")


The most frequent value is: [[1072, 648]]


In [45]:
# 1 - Data Loading and Preprocessing:
# Set up data loaders to load and preprocess your image data:

# Define data directories
train_data_dir = 'Dataset/train'
test_data_dir = 'Dataset/test'

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((modes[0][0], modes[0][1])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = ImageFolder(root=train_data_dir, transform=transform)
test_dataset = ImageFolder(root=test_data_dir, transform=transform)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Define the number of classes
num_classes = len(train_dataset.classes)


# CNN

In [48]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, num_classes, input_size):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Calculate the size of the final fully connected layer based on input_size
        self.final_fc_size = 32 * (input_size[0] // 4) * (input_size[1] // 4)
        self.classifier = nn.Sequential(
            nn.Linear(self.final_fc_size, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Initialize the CNN model with the desired input size
input_size = (modes[0][0], modes[0][1])
# Initialize the model
model = CNN(num_classes, input_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [49]:
# Training loop
num_epochs = 3  # Adjust as needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {loss.item()}")

print("Training finished.")


Epoch [1/10] - Loss: 0.09708212316036224
Epoch [2/10] - Loss: 0.05024762079119682
Epoch [3/10] - Loss: 0.01573743112385273
Epoch [4/10] - Loss: 0.010693574324250221
Epoch [5/10] - Loss: 0.00020770782430190593


KeyboardInterrupt: 

In [50]:
# Testing loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 75.16%


# ResNet-50 model

In [ ]:
# Check if folder existes else creates it
def create_folder_if_not_exists(folder_path):

    if not os.path.exists(folder_path):
        try:
            os.makedirs(folder_path)
            print(f"Folder '{folder_path}' created.")
        except OSError as e:
            print(f"Error creating folder '{folder_path}': {e}")

In [ ]:
# Load a pre-trained ResNet-50 model
model = torchvision.models.resnet50(pretrained=True)

# Freeze all layers except the final classification layer
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad = True


# Modify the final classification layer to match your number of classes
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(2048, num_classes)  # 2048 is the output size of ResNet-50


# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Train the model
num_epochs = 3 #20 - Loss: 0.2588 -acc:83.78%
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

print("Training finished.")


In [ ]:
# Evaluate the model on the test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test data: {accuracy:.2f}%")


if accuracy>75:
    # create Models
    create_folder_if_not_exists('Models/')
    # Save the trained model to a file
    torch.save(model, 'Models/ResNet-50.model.pth')